In [ ]:
# imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from math import sqrt

from pydataset import data

from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report,confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import export_graphviz
from sklearn.feature_selection import f_regression 

import graphviz
from graphviz import Graph

import env
import acquire
import prepare
import os

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# Planning

**The goal** of this stage is to clearly define your goal(s), measures of success, and plans on how to achieve that.

**The deliverable** is documentation of your goal, your measure of success, and how you plan on getting there.

**How to get there:** You can get there by answering questions about the final product & formulating or identifying any initial hypotheses (from you or others).

**Common questions include:**
- What will the end product look like?
- What format will it be in?
- Who will it be delivered to?
- How will it be used?
- How will I know I'm done?
- What is my MVP?
- How will I know it's good enough?


**Formulating hypotheses**
- Is attribute V1 related to attribute V2?
- Is the mean of target variable Y for subset A significantly different from that of subset B?

# Acquisition

**The goal** is to create a path from original data sources to the environment in which you will work with the data. You will gather data from sources in order to prepare and clean it in the next step.

**The deliverable** is a file, acquire.py, that contains the function(s) needed to reproduce the acquisition of data.

**How to get there:**

- If the data source is SQL, you may need to do some clean-up, integration, aggregation or other manipulation of data in the SQL environment before reading the data into your python environment.
- Using the Python library pandas, acquire the data into a dataframe using a function that reads from your source type, such as pandas.read_csv for acquiring data from a csv.
- You may use Spark and/or Hive when acquiring data from a distributed environment, such as HDFS.
Examples of source types include RDBMS, NoSQL, HDFS, Cloud Files (S3, google drive), static local flat files (csv, txt, xlsx).

In [ ]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    This function takes in user credentials from an env.py file and a database name and creates a connection to the Codeup database through a connection string 
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
    zillow_sql_query =  '''
                    select *
                    from properties_2017
                    join predictions_2017 using(parcelid)
                    join propertylandusetype using(propertylandusetypeid)
                    where propertylandusedesc = 'Single Family Residential'
                    and transactiondate like '2017%%';
                    '''
    
def query_zillow_data():
    '''
    This function uses the get_connection function to connect to the zillow database and returns the zillow_sql_query read into a pandas dataframe
    '''
    return pd.read_sql(zillow_sql_query,get_connection('zillow'))


def get_zillow_data():
    '''
    This function checks for a local zillow.csv file and reads it into a pandas dataframe, if it exists. If not, it uses the get_connection & query_zillow_data functions to query the data and write it locally to a csv file
    '''
    # If csv file exists locally, read in data from csv file.
    if os.path.isfile('zillow.csv'):
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Query and read data from zillow database
        df = query_zillow_data()
        
        # Cache data
        df.to_csv('zillow.csv')
        
    return df

In [ ]:
pd.read_clipboard
pd.read_excel
pd.read_csv
pd.

# Preparation

**The goal** is to have data, split into 3 samples (train, validate, and test), in a format that can easily be explored, analyzed and visualized. 

**The deliverable** is a file, prep.py, that contains the function(s) needed to reproduce the preparation of the data.

**How to get there:**

- Python libraries: pandas, matplotlib, seaborn, scikit-learn.
- Use pandas to perform tasks such as handling null values, outliers, normalizing text, binning of data, changing data types, etc.
- Use matplotlib or seaborn to plot distributions of numeric attributes and target.
- Use scikit-learn to split the data into train and test samples.

In [ ]:
df.head()
df.columns
df.index
df.shape
df.describe().T
df.info()
df.dtypes
# document takeaways

## Step 1: Remove unwanted observations (duplicate, irrelevant)

In [ ]:
df.drop(columns=['column1', 'column2'])
df.drop_duplicates(inplace=True)

## Step 2: Tidy data
* change data types, correct and standardize text,

In [ ]:
df[col].astype(int)
df = df.replace(r'^\s*$', np.nan, regex=True)

## Step 3: Take Care of Outliers
* ignore, drop rows, snap to a selected max/min value, create bins (cut, qcut)

In [ ]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    for col in col_list:
        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

df = remove_outliers(df, 1.5, ['col1', 'col2', 'col3'])

## Step 4: Handle Missing Values
* drop columns or rows with missing values
* fill with zero or other value where it makes sense
* impute values (must be done on split data)

In [ ]:
# checking for and handle nulls
df.isnull().sum()
df[df.exam1.isna()]
df.isnull().any()
df.fillna(value='value')
# How many nulls have in each row?
df.isnull().sum(axis =1).value_counts()
# impute 
imputer = SimpleImputer(missing_values = None, strategy='most_frequent') # mean, median, or most frequent
imputer = imputer.fit(train[['col']])
train[['col']] = imputer.transform(train[['col']])
validate[['col']] = imputer.transform(validate[['col']])
test[['col']] = imputer.transform(test[['col']])

In [ ]:
def impute_mode(train, validate, test, cols, strategy='most_frequent'):
    '''
    This formula takes in train, validate, and test dataframes, imputes the value specified,
    and returns train, validate, and test dataframes
    '''
    imputer = SimpleImputer(missing_values = None, strategy=strategy)
    train[[cols]] = imputer.fit_transform(train[[cols]])
    validate[[cols]] = imputer.transform(validate[[cols]])
    test[[cols]] = imputer.transform(test[[cols]])
    return train, validate, test

## Step 5: Rename Columns

In [ ]:
df.rename(columns={'name': 'student'})

## Step 6: Feature Engineering

In [ ]:
numbers_series.apply(lambda n: 'even' if n % 2 == 0 else 'odd')

## Step 7: Encode Columns

In [ ]:
dummy_df = pd.get_dummies(df[['col1','col2']], dummy_na=False, drop_first=[True, True])
df = pd.concat([df, dummy_df], axis=1)

## Step 8: Split Data

In [ ]:
# Split the data into train, validate, and test
def split_data(df, random_state=369, stratify=None):
    '''
    This function takes in a dataframe and splits the data into train, validate and test samples. 
    Test, validate, and train are 20%, 24%, & 56% of the original dataset, respectively. 
    The function returns train, validate and test dataframes.
    '''
    if stratify == None:
        # split dataframe 80/20
        train_validate, test = train_test_split(df, test_size=.2, random_state=random_state)
        # split larger dataframe from previous split 70/30
        train, validate = train_test_split(train_validate, test_size=.3, random_state=random_state)
    else:
        # split dataframe 80/20
        train_validate, test = train_test_split(df, test_size=.2, random_state=random_state, stratify=df[stratify])
        # split larger dataframe from previous split 70/30
        train, validate = train_test_split(train_validate, test_size=.3, 
                            random_state=random_state,stratify=train_validate[stratify])
    # results in 3 dataframes
    return train, validate, test

## Step 9: Scale Data

# Exploration & Pre-Processing

**The goal** is to discover features that have the largest impact on the target variable, i.e. provide the most information gain, drive the outcome.

**The deliverable** is a file, preprocess.py, that contains the function(s) needed to reproduce the pre-processing of the data. 

The dataframe resulting from these functions should be one that is pre-processed, i.e. ready to be used in modeling. This means that attributes are reduced to features, features are in a numeric form, there are no missing values, and continuous and/or ordered values are scaled to be unitless.

**How to get there:**

- Use python libraries: pandas, statsmodels, scipy, numpy, matplotlib, seaborn, scikit-learn.
- Perform statistical testing to understand correlations, significant differences in variables, variable interdependencies, etc.
- Create visualizations that demonstrate relationships across and within attributes and target.
- Use domain knowledge and/or information gained through exploration to construct new features.
- Remove features that are noisy, provide no valuable or new information, or are redundant.
- Use scikit-learn's preprocessing algorithms (feature selection, feature engineering, dummy variables, binning, clustering, e.g.) to turn attributes into features.

## Step 1: Explore Univariate Stats

In [ ]:
# univariate stats

# to prevent scientific notation where avoidable
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))
# has option to include or exclude certain dtypes

df.col.value_counts(dropna=False, ascending=True).plot.bar()

df.groupby('col').col.describe()
df[['col1', 'col2', 'col3']].agg(['mean', 'min', 'max'])
df.groupby('col').col.agg(['min', 'mean', 'max'])

# Use .describe with object columns.
obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True, dropna=False))
    print('----------------------')
    
# Check out distributions of numeric columns.
num_cols = df.columns[[df[col].dtype == 'int64' for col in df.columns]]
for col in num_cols:
    plt.hist(df[col])
    plt.title(col)
    plt.show()

## Step 2: Ask questions before exploring further

## Step 3: Explore bivariate stats

In [ ]:
# scatterplot, catplot, barplot, boxplot
# bin data

In [ ]:
plt.figure(figsize=(14, 6)) # (width, height)
plt.plot(x1, c='green', alpha=0.6)
plt.plot(x2, c='red', alpha=0.4)
plt.xlim(-20, 200)
plt.ylim(-5, 160)
plt.title('A couple of random series')
plt.legend(loc='upper right')
plt.xlabel('$x$')
plt.ylabel('$x^2$')
plt.xticks([1, 2, 3.14, 4], ['one', 'two', '$\pi$', 'four'], rotation=45)
plt.text(0.25, 0, '(0, 0)', fontsize=10, color='blue')
plt.annotate('Intersection', xy=(0, 0), xytext=(-3, 5),
             arrowprops={'facecolor': 'blue'})
population_survival_rate = train.survived.mean()
plt.axhline(population_survival_rate, label="Population survival rate")
# plot the first subplot
plt.subplot(n_rows, n_cols, 1)
plt.suptitle('Subplots Demo')
# Until we call .show, we'll be referring to the same chart, so we can keep adding to it.
plt.show()
plt.savefig('my-figure')

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = ['exam1', 'exam2', 'exam3', 'final_grade']

for i, col in enumerate(cols):
    
    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 
    
    # Create subplot.
    plt.subplot(1,4, plot_number)
    
    # Title with column name.
    plt.title(col)
    
    # Display histogram for column.
    df[col].hist(bins=5, edgecolor='black')
    
    # Hide gridlines.
    plt.grid(False)

In [ ]:
# visualize
sns.displot(x='final_grade', data=df)

plt.title('final_grade')
plt.show()


sns.pairplot(df, corner=True)

In [ ]:
data = pd.crosstab(tips.time, tips['size'])
sns.heatmap(data, annot=True, cmap=plt.cm.Greens)
data

In [ ]:
# crosstab to view relationship between two variables
pd.crosstab(X_train.contract_type, y_train, normalize='index', margins=True).style.background_gradient(cmap='Greens')

In [ ]:
df.pivot_table(index='classroom', columns='passing_math', values='math')

In [ ]:
# check assumptions: Normal distribution? Independent? Equal variance?

# compare observed mean to theoretical mean, numeric to categoricalb
t, p = stats.ttest_1samp(sample, overall_mean) # parametric, normal distribution
t, p = stats.wilcoxen(sample, overall_mean) # nonparametric
# compare two observed means (independent samples)
t, p = stats.ttest_ind(sample1, sample2, equal_var=False) # parametric, normal distribution
t, p = stats.mannwhitneyu(sample1, sample2) # nonparametric
# compare several observed means (independent samples)
f, p = stats.f_oneway(sample1, sample2, sample3) # parametric, normal distribution
stats.kruskal(sample1, sample2, sample3) # nonparametric
# check variance
df[col].var()

# check numeric to numeric relationship
corr, p = stats.pearsonr(x, y) # for linear relationship
corr, p = stats.spearmanr(x, y) # for nonlinear relationship

# compare two categorical variables
chi2, p, degf, expected = stats.chi2_contingency(observed) # observed is a crosstab

# "There is sufficient evidence to move forward understanding that there is a relationship between x and y"

if p < alpha:
    print("We reject $H_{0}$")
else:
    print("We fail to reject $H_{0}$")

## Document findings and takeaways even if it's that there is nothing between x and y

# Modeling

**The goal** is to create a robust and generalizable model that is a mapping between features and a target outcome.

**The deliverable** is a file, model.py, that contains functions for training the model (fit), predicting the target on new data, and evaluating results.

**How to get there:**

- Python libraries: scikit-learn
- Identify regression, classification, cross validataion, and/or other algorithms that are most appropriate.
- Build your model:
- Create the model object.
- Fit the model to your training, or in-sample, observations.
- Predict the target value on your training observations.
- Evaluate results on the in-sample predictions.
- Repeat as necessary with other algorithms or hyperparameters.
- Using the best performing model, predict on test, out-of-sample, observations.
- Evaluate results on the out-of-sample predictions.

In [ ]:
X_train = train.drop(columns=['cols'])
y_train = train.col

X_validate = validate.drop(columns=['cols'])
y_validate = validate.col

X_test = test.drop(columns=['cols'])
y_test = test.col

In [ ]:
# decision tree classifier
clf = DecisionTreeClassifier(max_depth=3, random_state=123)
clf = clf.fit(X_train, y_train)
# visualize decision tree
plt.figure(figsize=(13, 7))
plot_tree(clf, feature_names=X_train.columns, class_names=clf.classes_, rounded=True)
# make predictions
y_pred = clf.predict(X_train)
y_pred[0:5]
# estimate probability
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))
confusion_matrix(y_train, y_pred)
print(classification_report(y_train, y_pred))

model1 = DecisionTreeClassifier(max_depth=2)
model1.fit(X_train[features], y_train)
accuracy = model1.score(X_validate[features], y_validate)
print(f'Model 1 Accuracy: {accuracy:.2%}')

In [ ]:
# classification algorithms

# used to predict binary outcomes
Logistic Regression (sklearn.linear_model.LogisticRegression)
# A sequence of rules used to classify 2 or more classes.
Decision Tree (sklearn.tree.DecisionTreeClassifier)
Naive Bayes (sklearn.naive_bayes.BernoulliNB)
K-Nearest Neighbors (sklearn.neighbors.KNeighborsClassifier)
Random Forest (sklearn.ensemble.RandomForestClassifier)
Support Vector Machine (sklearn.svm.SVC)
Stochastic Gradient Descent (sklearn.linear_model.SGDClassifier)
AdaBoost (sklearn.ensemble.AdaBoostClassifier)
Bagging (sklearn.ensemble.BaggingClassifier)
Gradient Boosting (sklearn.ensemble.GradientBoostingClassifier)

# Delivery

**The goal** is to enable others to use what you have learned or developed through all the previous stages.

**The deliverable** could be of various types:

- A pipeline.py file that takes new observations from acquisition to prediction using the previously built functions.
- A fully deployed model.
- A reproducible report and/or presentation with recommendations of actions to take based on original project goals.
- Predictions made on a specific set of observations.
- A dashboard for observing/monitoring the key drivers, or features, of the target variable.

**How to get there:**

- Python sklearn's pipeline method.
- Tableau for creating a report, presentation, story, or dashboard.
- Jupyter notebook for creating a report or a framework to reproduce your research, e.g.
- Flask to build a web server that provides a gateway to our model's predictions.